In [1]:
from bot import Bot
from game_config import Config, Blackjack
from shoe import Shoe
from tqdm import tqdm_notebook as tqdm

In [12]:
# data = {}
# for target in range(1,11):
#     data[target] = {}
#     for var1 in range(1,11):
#         data[target][var1] = {}
#         for var2 in range(1,11):
#             if var2 >= var1:
#                 data[target][var1][var2] = {}
#                 for act in ['hit','stand','x2','split']:
#                     data[target][var1][var2][act] = []

In [13]:
data

{1: {1: {1: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   2: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   3: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   4: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   5: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   6: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   7: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   8: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   9: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   10: {'hit': [], 'stand': [], 'x2': [], 'split': []}},
  2: {2: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   3: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   4: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   5: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   6: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   7: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   8: {'hit': [], 'stand': [], 'x2': [], 'split': []},
   9: {'hit': [], 'stand': [], 'x2': [], 'split': []},

In [ ]:
class simul_log:
    def __init__(data):
        self.data = data
        
    

In [14]:
bet_amount = 1
bot_signup = []
for i in range(6):
    bot_signup.append(Bot(bankroll = 1_000_000, name = 'Player {}'.format(i+1)))

config = Config() # Define your configuration game here. Check game_config.py for reference

blackjack = Blackjack(config = config, player = bot_signup,
                      shoe = Shoe(deck_quant = config.n_deck))
card_quant = len(blackjack.shoe.shoe) # Total cards in the shoe

In [15]:
game = blackjack.run()

In [16]:
game.dealer.get_hand()

['02:H', '03:H']

In [17]:
[v.get_hand() for v in game.players]

[['10:C', 'Qe:S'],
 ['03:C', '04:D'],
 ['Ki:C', '02:S'],
 ['10:H', '09:D'],
 ['Qe:C', 'Ki:H'],
 ['05:D', '04:D']]

In [2]:
def simulate(n_loop: int = 1000, n_player: int = 1):
    
    bet_amount = 1
    bot_signup = []
    for i in range(n_player):
        bot_signup.append(Bot(bankroll = 1_000_000, name = 'Player {}'.format(i+1)))
        
    config = Config() # Define your configuration game here. Check game_config.py for reference

    blackjack = Blackjack(config = config, player = bot_signup,
                          shoe = Shoe(deck_quant = config.n_deck))
    card_quant = len(blackjack.shoe.shoe) # Total cards in the shoe

    loop = 0
    while loop < n_loop:
        if len(blackjack.shoe.shoe) < int(card_quant*blackjack.config.reset_shoe):
            blackjack.shoe.shuffle(reset=True)
            
        game = blackjack.run()

        dealer_hand = game.dealer
        bot = game.players[0]

        faceup_value = str(dealer_hand.get_faceup_card().get_score()[0])
        bot_value = sorted([card.get_score()[0] for card in bot.hand], reverse=True)

        if faceup_target:
            if faceup_target != faceup_value:
                continue
                
        if hand_target:
            if sorted(hand_target, reverse=True) != bot_value:
                continue

        # Processing block
        for bot_hand in [bot]: # Player-scanning Block
            hand_process = [bot_hand]
            hand_count = 0

            while len(hand_process) != 0: # All-hands loop Block
                hand_in_play = hand_process[0]
                _count = 0

                quest = game.question(hand_in_play, _count, insurance_offer = False)
                if hand_count != 0:
                    quest = 'Hand {} --- {}'.format(hand_count,quest)

                while quest != None: # Response Block
                    # Bot action
                    if action_target and hand_count == 0 and _count == 0:
                        action = action_target
                        if action_target == 'split' and hand_in_play.split_check() == False:
                            break
                    else:
                        action = hand_in_play.player.play(faceup_card = game.dealer.get_faceup_card(), hand = hand_in_play, question = quest)
                    _count += 1

                    if action == 'x2':
                        hand_in_play.x2(blackjack.shoe)
#                         print(hand_in_play.get_hand())
                    elif action == 'hit':
                        hand_in_play.hit(blackjack.shoe)
#                         print(hand_in_play.get_hand())
                    elif action == 'stand':
                        hand_in_play.stand()
                    elif action == 'split':
                        hand_1, hand_2 = hand_in_play.split(blackjack.shoe)
                        hand_process += [hand_1, hand_2]
                        # hand_delete = True
                        break

                    quest = game.question(hand_in_play, _count)
#                     if quest != None:
# #                         print('Quest: {}'.format(quest))

                hand_process.pop(0)

                hand_count += 1
        
        game.dealer_action(blackjack.shoe) 
#         print('-----------------------------------------------')
#         print(hand_print(dealer_hand, player=False))

        for bot_hand in [bot]: # Player-scanning Score block
#             print('Bot {} ------ Results: '.format(bot_hand.player.name))
            hands = game.get_hands(player_name = bot_hand.player.name)
            for hand in hands:
#                 print(hand_print(hand))
                game.process_result(hand, bet_amount, printout=False)

        blackjack.log(game)

        loop += 1
        
        pbar.update(1)

    return blackjack.logs
    